In [230]:
#!/usr/bin/python3
import pandas as pd
import json

import influxdb_client
from influxdb import DataFrameClient
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

from dateutil import parser
import requests
from string import Template


In [231]:
# Use only following fields from CSV. 
Fields = ['TimeStamp','School','Class','Students','Absent']
# Define tag fields
datatags = ['School','Class']
# Define fixed tags
fixtags = {"Country": "India", "State": "Haryana", "City": "Kurukshetra"}

# Read data from CSV without index and parse 'TimeStamp' as date.
df = pd.read_csv("sample.csv", sep=';', index_col=False, parse_dates=['TimeStamp'], usecols=Fields)

# Set 'TimeStamp' field as index of dataframe
df.set_index('TimeStamp', inplace = True)

print(df.head())

                              School Class  Students  Absent
TimeStamp                                                   
2020-05-09 17:45:00+00:00      USSMS   UKG        30       3
2020-05-09 17:45:00+00:00      USSMS    10        50       2
2020-05-09 17:45:00+00:00      USSMS     8        40       0
2020-05-09 18:45:00+00:00  St Thomas   Nur        30       4
2020-05-09 18:45:00+00:00  St Thomas     6        55       4


In [232]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

bucket = "DSI_test"
token ="yxk8_Or5qHZJxrhJE3SkAnTQSViQCsmrUoR0xPZd_0scy1T8FTuL1cKSTDKh1ft8Bqs3Zbt7Rwkys-FzajIVFQ=="
org="DSI"
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)

In [233]:
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

In [234]:

from influxdb_client.client.write_api import ASYNCHRONOUS

records = pd.DataFrame(
    data=[
        ["0", 25.3, 1657729063],
    	  ["0", 25.4, 1657729078],
    	  ["0", 25.2, 1657729093],
    ],
    columns=["core", "temp", "timestamp"],
)
records["timestamp"] = pd.to_datetime(records["timestamp"], unit="s")
records.set_index("timestamp", inplace=True)
#records.drop(colums=["core"], inplace=True)

with client.write_api(write_options=ASYNCHRONOUS) as writer:
    writer.write(
        bucket=bucket,
        record=records,
        data_frame_measurement_name="cpu3",
        data_frame_tag_columns=["core"]
        )

In [235]:
with client.write_api(write_options=ASYNCHRONOUS) as writer:
    writer.write(
        bucket=bucket,
        record=df,
        data_frame_measurement_name="student",
        data_frame_tag_columns=datatags,
        data_frame_field_columns=["Students", "Absent"]
        )


In [236]:
#Data Source

import yfinance as yf



#Interval required 1 minute

data = yf.download(tickers='IBM', period='1d', interval='1m')

[*********************100%***********************]  1 of 1 completed


In [237]:
data = data.to_json(orient='index')

In [238]:
#make data to dataframe
df_IBM = pd.read_json(data).T
df_IBM

,Open,High,Low,Close,Adj Close,Volume
2023-01-20 14:30:00,141.630005,141.630005,141.630005,141.630005,141.630005,408021.0
2023-01-20 14:31:00,141.580002,141.789993,141.419006,141.649994,141.649994,20861.0
2023-01-20 14:32:00,141.604996,141.604996,141.250000,141.350006,141.350006,9920.0
2023-01-20 14:33:00,141.297699,141.429993,141.210007,141.250000,141.250000,8221.0
2023-01-20 14:34:00,141.270004,141.320007,141.100006,141.304993,141.304993,9136.0
...,...,...,...,...,...,...
2023-01-20 18:14:00,140.770004,140.850006,140.770004,140.792007,140.792007,9816.0
2023-01-20 18:15:00,140.779999,140.779999,140.740005,140.750000,140.750000,31852.0
2023-01-20 18:16:00,140.800003,140.804993,140.770004,140.779999,140.779999,4285.0
2023-01-20 18:17:00,140.759995,140.759995,140.750000,140.759995,140.759995,3524.0


In [239]:
# symbols = ['IBM']

# url = Template('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=$symbol&interval=5min&apikey=5LZM13OZQMZFPU92')

# for symbol in symbols:
#     r = requests.get(url.substitute(symbol=symbol))
#     data = r.json()

#     #print(data)

# #make json to dataframe

# df_IBM = pd.DataFrame(data['Time Series (5min)']).T


In [240]:
#df_IBM.columns = ['open', 'high', 'low', 'close', 'volume']

#change col types to float

df_IBM = df_IBM.astype(float)


In [241]:
with client.write_api(write_options=ASYNCHRONOUS) as writer:
    writer.write(
        bucket=bucket,
        record=df_IBM,
        data_frame_measurement_name="IBM_3",
        data_frame_tag_columns=["symbol"],
        data_frame_field_columns=["open", "high", "low", "close", "volume"]
        )

In [242]:
df_IBM

,Open,High,Low,Close,Adj Close,Volume
2023-01-20 14:30:00,141.630005,141.630005,141.630005,141.630005,141.630005,408021.0
2023-01-20 14:31:00,141.580002,141.789993,141.419006,141.649994,141.649994,20861.0
2023-01-20 14:32:00,141.604996,141.604996,141.250000,141.350006,141.350006,9920.0
2023-01-20 14:33:00,141.297699,141.429993,141.210007,141.250000,141.250000,8221.0
2023-01-20 14:34:00,141.270004,141.320007,141.100006,141.304993,141.304993,9136.0
...,...,...,...,...,...,...
2023-01-20 18:14:00,140.770004,140.850006,140.770004,140.792007,140.792007,9816.0
2023-01-20 18:15:00,140.779999,140.779999,140.740005,140.750000,140.750000,31852.0
2023-01-20 18:16:00,140.800003,140.804993,140.770004,140.779999,140.779999,4285.0
2023-01-20 18:17:00,140.759995,140.759995,140.750000,140.759995,140.759995,3524.0
